In [27]:
import argparse
import math
import sys
import time
import copy

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, BatchNormalization, regularizers
from keras.layers.noise import GaussianNoise
from keras.layers import Conv1D, MaxPooling1D
from keras import backend as K
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils.np_utils import to_categorical
K.set_image_dim_ordering('th')
print(K.image_data_format())

## required for efficient GPU use
import tensorflow as tf
from keras.backend import tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
session = tf.Session(config=config)
tensorflow_backend.set_session(session)
## required for efficient GPU use

import os
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

channels_first


In [0]:
model_path='cnn_deep_mon.h5'


callbacks = [
    EarlyStopping(
        monitor='val_acc', 
        patience=10,
        mode='max',
        verbose=1),
    ModelCheckpoint(model_path,
        monitor='val_acc', 
        save_best_only=True, 
        mode='max',
        verbose=0)
]

In [29]:
class dataset:
    monday = pd.read_pickle("preprocessed_monday_data.pkl")
    tuesday = pd.read_pickle("preprocessed_thursday_data.pkl")
    
    
from sklearn import model_selection as ms
from sklearn import preprocessing as pp

class preprocess:
    
    output_columns_2labels = ['label_0','label_1']
    
    x_input = dataset.monday.drop(output_columns_2labels, axis = 1)
    y_output = dataset.monday.loc[:,output_columns_2labels]

    x_test_input = dataset.tuesday.drop(output_columns_2labels, axis = 1)
    y_test = dataset.tuesday.loc[:,output_columns_2labels]
    
    ss = pp.StandardScaler()
    
    x_train = ss.fit_transform(x_input)
    x_test = ss.fit_transform(x_test_input)
    
    y_train = y_output.values
    y_test = y_test.values
        
    print('x_train shape: {}'.format(x_train.shape))
    print('y_train shape: {}'.format(y_train.shape))
    print('x_test shape: {}'.format(x_test.shape))
    print('y_test shape: {}'.format(y_test.shape))

x_train shape: (529918, 78)
y_train shape: (529918, 2)
x_test shape: (458968, 78)
y_test shape: (458968, 2)


In [30]:
X_train = np.reshape(preprocess.x_train, (preprocess.x_train.shape[0], preprocess.x_train.shape[1], 1))
X_test = np.reshape(preprocess.x_test, (preprocess.x_test.shape[0], preprocess.x_test.shape[1], 1))

print('train shape after reshape: {}'.format(X_train.shape))
print('test shape after reshape: {}'.format(X_test.shape))

train shape after reshape: (529918, 78, 1)
test shape after reshape: (458968, 78, 1)


In [0]:
#size of parameters
batch_size = 1000
num_classes = 2
epochs = 10
filter_size=3
#noise = 1
droprate=0.50

In [32]:
#Start Neural Network
model = Sequential()

#convolution 1st layer
model.add(Conv1D(64, kernel_size=(filter_size), padding="same",
                 activation='relu',
                 input_shape=(78, 1)))
model.add(BatchNormalization())
model.add(Dropout(droprate))
          
#convolution 2nd layer
model.add(Conv1D(128, kernel_size=(filter_size), activation='relu', border_mode="same"))
model.add(BatchNormalization())
model.add(MaxPooling1D(strides=1))
model.add(Dropout(droprate))

#convolution 3rd layer
model.add(Conv1D(256, kernel_size=(filter_size), activation='relu', border_mode="same"))
model.add(BatchNormalization())
model.add(MaxPooling1D(strides=1))
model.add(Dropout(droprate))

#FCN 1st layer
model.add(Flatten())
model.add(Dense(64,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN 2nd layer
model.add(Dense(32,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN 3rd layer
model.add(Dense(16,use_bias=False))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(droprate))

#FCN final layer
model.add(Dense(2))
model.add(Activation('softmax'))

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, kernel_size=3, activation="relu", padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, kernel_size=3, activation="relu", padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 78, 64)            256       
_________________________________________________________________
batch_normalization_13 (Batc (None, 78, 64)            256       
_________________________________________________________________
dropout_13 (Dropout)         (None, 78, 64)            0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 78, 128)           24704     
_________________________________________________________________
batch_normalization_14 (Batc (None, 78, 128)           512       
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 77, 128)           0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 77, 128)           0         
__________

In [33]:
#Save Model=ON
history = model.fit(X_train, preprocess.y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, preprocess.y_test),shuffle=True)

score = model.evaluate(X_test, preprocess.y_test, verbose=0)

#print loss and accuracy
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 529918 samples, validate on 458968 samples
Epoch 1/10
529918/529918 [==============================] - 109s 205us/step - loss: 0.0968 - acc: 0.9856 - val_loss: 0.0268 - val_acc: 0.9952
Epoch 2/10
 71000/529918 [===>..........................] - ETA: 1:15 - loss: 0.0152 - acc: 1.0000529918/529918 [==============================] - 106s 200us/step - loss: 0.0094 - acc: 1.0000 - val_loss: 0.0329 - val_acc: 0.9952
Epoch 3/10
264000/529918 [=============>................] - ETA: 43s - loss: 0.0045 - acc: 1.0000529918/529918 [==============================] - 106s 201us/step - loss: 0.0038 - acc: 1.0000 - val_loss: 0.0415 - val_acc: 0.9952
Epoch 4/10
337000/529918 [==================>...........] - ETA: 31s - loss: 0.0024 - acc: 1.0000529918/529918 [==============================] - 106s 201us/step - loss: 0.0022 - acc: 1.0000 - val_loss: 0.0468 - val_acc: 0.9952
Epoch 5/10
364000/529918 [===================>..........] - ETA: 27s - loss: 0.0015 - acc: 1.0000529918/529918 [=========

In [34]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = model.predict(X_test)
print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
print(y_pred)

y_pred = model.predict_classes(X_test)
print(y_pred)

p=model.predict_proba(X_test)

target_names = ['0', '1']
print(classification_report(np.argmax(preprocess.y_test, axis=1), y_pred, target_names=target_names, digits=4))
print(confusion_matrix(np.argmax(preprocess.y_test, axis=1), y_pred))

[[9.9999654e-01 3.4643538e-06]
 [9.9999666e-01 3.2939658e-06]
 [9.9999988e-01 6.3009459e-08]
 ...
 [9.9999988e-01 8.6628184e-08]
 [1.0000000e+00 5.6835674e-08]
 [1.0000000e+00 5.1025747e-08]]
[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

          0     0.9952    1.0000    0.9976    456752
          1     0.0000    0.0000    0.0000      2216

avg / total     0.9904    0.9952    0.9928    458968

[[456752      0]
 [  2216      0]]


In [26]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving preprocessed_thursday_data.pkl to preprocessed_thursday_data.pkl
User uploaded file "preprocessed_thursday_data.pkl" with length 287318820 bytes
